In [ ]:
# Set backend for Keras to be Theano
# You can make multiple Keras models this way w/o slowing down, unlike Tensorflow

from keras import backend as K
import os
from importlib import reload

def set_keras_backend(backend):

    if K.backend() != backend:
        os.environ['KERAS_BACKEND'] = backend
        reload(K)
        assert K.backend() == backend

set_keras_backend("theano")

In [ ]:
cd ~/nnet-survival/

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from keras.models import Sequential, Model
from keras import optimizers, layers, regularizers
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import Input, Dense, Activation
from keras.models import load_model
from lifelines import KaplanMeierFitter
from lifelines import CoxPHFitter
from lifelines.utils import concordance_index
import nnet_survival

In [ ]:
import numpy as np
import pandas as pd

from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout
from keras.optimizers import SGD, RMSprop
import theano.tensor as T

from lifelines.utils import concordance_index
from lifelines import CoxPHFitter

from lifelines.datasets import load_rossi
from sklearn import preprocessing

In [ ]:
# Get disease codes for each disease type that we want to make comparisons between baseline/transfer

df2 = pd.read_csv('/home/nolelin/clinical3.csv')
df2 = df2.loc[df2['name'] == 'disease_code']
disease_codes = list(set(list(df2['value'])))

In [ ]:
# Get all case_ids of patients for each disease type

disease_caseids = {}
df2 = pd.read_csv('/home/nolelin/clinical3.csv')
for disease in disease_codes:
    diseasedf = df2.loc[df2['value'] == disease]
    disease_caseids[disease] = list(diseasedf['case_id'])

In [ ]:
# Only train/validate on 80% of all patients
# Do 10 runs

from random import shuffle

runs_constrained_caseids = {}
for run in range(10):
    constrained_caseids = []
    for disease in disease_caseids.keys():
        num_patients = len(disease_caseids[disease])
        shuffle(disease_caseids[disease])
        constrained_caseids.extend(disease_caseids[disease][0:int(0.8 * num_patients)])
    runs_constrained_caseids[run] = constrained_caseids

In [ ]:
# Get best mutation feature indices

best_mut_feat_dict = np.load('/home/nolelin/best_mut_feats.npy', encoding='latin1').item()
best_mut_feats = sorted(best_mut_feat_dict.keys())

In [ ]:
# Clean and preprocess data for global model

import numpy as np
import sys
import numpy.ma as ma
from sklearn import preprocessing
from sklearn.impute import SimpleImputer

runs_x_train = {}
runs_x_test = {}
runs_x_val = {}
runs_y_train = {}
runs_y_test = {}
runs_y_val = {}
runs_e_train = {}
runs_e_test = {}
runs_e_val = {}
runs_n_intervals = {}
runs_breaks = {}
    
for run in range(10):
    print(run)
    disease_dfs = []
    for disease in disease_codes:
        if disease == 'FPPP':
            continue
        df = pd.read_csv('/mnt/raid/cpg_fs_checkpoints/meth_mut_exp_features/df.csv')[['features', 'days_to_death', 'days_to_followup', 'case_id', 'mut_features', 'days_to_birth', 'exp_features']]
        df = df.dropna(subset=['days_to_death', 'days_to_followup'], how='all')
        df = df.loc[df['case_id'].isin(disease_caseids[disease])]
        df['disease_code'] = disease
        disease_dfs.append(df)
    df = pd.concat(disease_dfs)
    df = pd.get_dummies(df, columns=['disease_code'])
    disease_cols = [x for x in df.columns if 'disease_code' in x]
    
    output_cols = ['days_to_death', 'days_to_followup']
    imp = SimpleImputer(missing_values=np.nan, strategy='mean')
    x = np.asarray([[float(y) if y != 'NA' else np.nan for y in x[1:-1].split(", ")] for x in list(df['features'])])
    x_mut = np.asarray([[float(y) if y != 'NA' else np.nan for y in x[1:-1].split(", ")] for x in list(df['mut_features'])])
    x_exp = np.asarray([[float(y) if y != 'NA' else np.nan for y in x[1:-1].split(", ")] for x in list(df['exp_features'])])
    mut_load = np.sum(x_mut, axis=1).reshape(df.shape[0], 1)
    x = np.hstack((x, mut_load))
    x = np.hstack((x, np.expand_dims(np.asarray(df['days_to_birth']), axis=1)))
    x = np.hstack((x, x_exp))
    x = np.where(np.isnan(x), ma.array(x, mask=np.isnan(x)).mean(axis=0), x)
    imp.fit(x)
    x = imp.transform(x)
    scaler = preprocessing.StandardScaler().fit(x)
    days = np.asarray(df[output_cols])
    y_whole = np.asarray([x[0] if np.isnan(x[0]) == False else x[1] for x in days])
    breaks=np.arange(min(y_whole), max(y_whole), 200)
    n_intervals=len(breaks)-1
    
    test = df.loc[~df['case_id'].isin(runs_constrained_caseids[run])]
    df = df.loc[df['case_id'].isin(runs_constrained_caseids[run])]
    df = df.sample(frac=1)
    train_size = int(0.75 * df.shape[0])
    test_size = df.shape[0] - train_size
    train = df.head(train_size)
    validate = df.tail(test_size)
    dtb_train = np.expand_dims(np.asarray(train['days_to_birth']), axis=1)
    mut_train = np.asarray([[float(y) if y != 'NA' else np.nan for y in x[1:-1].split(", ")] for x in list(train['mut_features'])])
    mut_train_load = np.sum(mut_train, axis=1).reshape(train.shape[0], 1)
    mut_train = mut_train[:, best_mut_feats]
    exp_train = np.asarray([[float(y) if y != 'NA' else np.nan for y in x[1:-1].split(", ")] for x in list(train['exp_features'])])
    x_train = np.asarray([[float(y) if y != 'NA' else np.nan for y in x[1:-1].split(", ")] for x in list(train['features'])])
    x_train = np.hstack((x_train, mut_train_load))
    x_train = np.hstack((x_train, dtb_train))
    x_train = np.hstack((x_train, exp_train))
    x_train = imp.transform(x_train)
    x_train = scaler.transform(x_train)
    x_train = x_train[:, ~np.all(x_train == 0, axis=0)]
    x_train = np.hstack((x_train, mut_train))
    x_train = np.hstack((x_train, train[disease_cols]))
    days = np.asarray(train[output_cols])
    y_train = np.asarray([x[0] if np.isnan(x[0]) == False else x[1] for x in days])
    e_train = np.asarray([1 if np.isnan(x[0]) == False else 0 for x in days])
    y_train=nnet_survival.make_surv_array(y_train,[True if x == 1 else False for x in e_train],breaks)
        
    dtb_test = np.expand_dims(np.asarray(test['days_to_birth']), axis=1)
    mut_test = np.asarray([[float(y) if y != 'NA' else np.nan for y in x[1:-1].split(", ")] for x in list(test['mut_features'])])
    mut_test_load = np.sum(mut_test, axis=1).reshape(test.shape[0], 1)
    mut_test = mut_test[:, best_mut_feats]
    exp_test = np.asarray([[float(y) if y != 'NA' else np.nan for y in x[1:-1].split(", ")] for x in list(test['exp_features'])])
    x_test = np.asarray([[float(y) if y != 'NA' else np.nan for y in x[1:-1].split(", ")] for x in list(test['features'])])
    x_test = np.hstack((x_test, mut_test_load))
    x_test = np.hstack((x_test, dtb_test))
    x_test = np.hstack((x_test, exp_test))
    x_test = imp.transform(x_test)
    x_test = scaler.transform(x_test)
    x_test = x_test[:, ~np.all(x_test == 0, axis=0)]
    x_test = np.hstack((x_test, mut_test))
    x_test = np.hstack((x_test, test[disease_cols]))
    days = np.asarray(test[output_cols])
    y_test = np.asarray([x[0] if np.isnan(x[0]) == False else x[1] for x in days])
    e_test = np.asarray([1 if np.isnan(x[0]) == False else 0 for x in days])
    y_test=nnet_survival.make_surv_array(y_test,[True if x == 1 else False for x in e_test],breaks)
    
    dtb_val = np.expand_dims(np.asarray(validate['days_to_birth']), axis=1)
    mut_val = np.asarray([[float(y) if y != 'NA' else np.nan for y in x[1:-1].split(", ")] for x in list(validate['mut_features'])])
    mut_val_load = np.sum(mut_val, axis=1).reshape(validate.shape[0], 1)
    mut_val = mut_val[:, best_mut_feats]
    exp_val = np.asarray([[float(y) if y != 'NA' else np.nan for y in x[1:-1].split(", ")] for x in list(validate['exp_features'])])
    x_val = np.asarray([[float(y) if y != 'NA' else np.nan for y in x[1:-1].split(", ")] for x in list(validate['features'])])
    x_val = np.hstack((x_val, mut_val_load))
    x_val = np.hstack((x_val, dtb_val))
    x_val = np.hstack((x_val, exp_val))
    x_val = imp.transform(x_val)
    x_val = scaler.transform(x_val)
    x_val = x_val[:, ~np.all(x_val == 0, axis=0)]
    x_val = np.hstack((x_val, mut_val))
    x_val = np.hstack((x_val, validate[disease_cols]))
    days = np.asarray(validate[output_cols])
    y_val = np.asarray([x[0] if np.isnan(x[0]) == False else x[1] for x in days])
    e_val = np.asarray([1 if np.isnan(x[0]) == False else 0 for x in days])
    y_val=nnet_survival.make_surv_array(y_val,[True if x == 1 else False for x in e_val],breaks)
    
    print(x_train.shape, x_val.shape, x_test.shape)
        
    if x_train.shape[1] != x_val.shape[1] or x_train.shape[1] != x_test.shape[1] or x_val.shape[1] != x_test.shape[1]:
        print("mismatched shape")
        continue

    if np.sum(e_train) < 0.2 * len(e_train) or np.sum(e_val) < 0.2 * len(e_val) or np.sum(e_test) < 0.2 * len(e_test) or len(e_train) < 150:
        print("skipping ", disease)
        continue
    
    runs_x_train[run] = x_train
    runs_x_test[run] = x_test
    runs_x_val[run] = x_val
    runs_y_train[run] = y_train
    runs_y_test[run] = y_test
    runs_y_val[run] = y_val
    runs_e_train[run] = e_train
    runs_e_test[run] = e_test
    runs_e_val[run] = e_val
    runs_n_intervals[run] = n_intervals
    runs_breaks[run] = breaks

In [ ]:
# Tuning for global models

from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Masking
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import optimizers
from keras import regularizers
import sys

runs_global_errors = {}
units = [10, 50, 100, 200, 500, 1000]
learning_rates = [0.1, 0.01, 0.001, 0.0001, 0.00001]
for run in range(10):
    for unit in units:
        for learning_rate in learning_rates:
            model = Sequential()
            model.add(Dense(unit, activation='relu', input_shape=(runs_x_train[run].shape[1],)))
            model.add(Dense(runs_n_intervals[run], activation='sigmoid'))
            adam = optimizers.Adam(lr=learning_rate)
            model.compile(loss=nnet_survival.surv_likelihood(runs_n_intervals[run]), optimizer=adam)
            early_stopping = EarlyStopping(monitor='val_loss', patience=2)
            history = model.fit(runs_x_train[run], runs_y_train[run], batch_size=256, epochs=1000, shuffle=True, callbacks=[early_stopping], validation_data=(runs_x_val[run], runs_y_val[run]))
            best_epochs = history.history['val_loss'].index(min(history.history['val_loss'])) + 1
            runs_global_errors[(run, (learning_rate, best_epochs, unit))] = min(history.history['val_loss'])

In [ ]:
# Train global models

runs_global_models = {}
for run in range(10):
    this_runs = [x for x in runs_global_errors.keys() if run == x[0]]
    lowest_loss = 10000
    best_run = None
    for this_run in this_runs:
        if runs_global_errors[this_run] < lowest_loss:
            lowest_loss = runs_global_errors[this_run]
            best_run = this_run
    print(best_run, runs_global_errors[best_run])
    total_x = np.vstack((runs_x_train[run], runs_x_val[run]))
    total_y = np.vstack((runs_y_train[run], runs_y_val[run]))
    total_e = np.hstack((runs_e_train[run], runs_e_val[run]))
    model = Sequential()
    model.add(Dense(best_run[1][2], activation='relu', input_shape=(total_x.shape[1],)))
    model.add(Dense(runs_n_intervals[run], activation='sigmoid'))
    adam = optimizers.Adam(lr=best_run[1][0])
    model.compile(loss=nnet_survival.surv_likelihood(runs_n_intervals[run]), optimizer=adam)
    model.fit(total_x, total_y, batch_size=256, epochs=best_run[1][1], shuffle=True)
    runs_global_models[run] = model

In [ ]:
# Do same data pre-processing for each individual disease
# Don't include diseases that don't have at least 150 patients in training set and at least 30% of patients have died

import numpy as np
import sys
import numpy.ma as ma
from sklearn import preprocessing
from sklearn.impute import SimpleImputer

runs_disease_x_train = {}
runs_disease_x_test = {}
runs_disease_x_val = {}
runs_disease_y_train = {}
runs_disease_y_test = {}
runs_disease_y_val = {}
runs_disease_y_test_orig = {}
runs_disease_e_train = {}
runs_disease_e_test = {}
runs_disease_e_val = {}
runs_disease_n_intervals = {}

for run in range(10):
    breaks = runs_breaks[run]
    n_intervals=len(breaks)-1
    for disease in disease_codes:
        if disease == 'FPPP':
            continue
        df = pd.read_csv('/mnt/raid/cpg_fs_checkpoints/meth_mut_exp_features/df.csv')[['features', 'days_to_death', 'days_to_followup', 'case_id', 'mut_features', 'days_to_birth', 'exp_features']]
        df = df.dropna(subset=['days_to_death', 'days_to_followup'], how='all')
        df = df.loc[df['case_id'].isin(disease_caseids[disease])]
        
        imp = SimpleImputer(missing_values=np.nan, strategy='mean')
        x = np.asarray([[float(y) if y != 'NA' else np.nan for y in x[1:-1].split(", ")] for x in list(df['features'])])
        x_mut = np.asarray([[float(y) if y != 'NA' else np.nan for y in x[1:-1].split(", ")] for x in list(df['mut_features'])])
        x_exp = np.asarray([[float(y) if y != 'NA' else np.nan for y in x[1:-1].split(", ")] for x in list(df['exp_features'])])
        mut_load = np.sum(x_mut, axis=1).reshape(df.shape[0], 1)
        x = np.hstack((x, mut_load))
        x = np.hstack((x, np.expand_dims(np.asarray(df['days_to_birth']), axis=1)))
        x = np.hstack((x, x_exp))
        x = np.where(np.isnan(x), ma.array(x, mask=np.isnan(x)).mean(axis=0), x)
        imp.fit(x)
        x = imp.transform(x)
        scaler = preprocessing.StandardScaler().fit(x)
        
        disease_idx = disease_cols.index('disease_code_' + disease)
        
        test = df.loc[~df['case_id'].isin(runs_constrained_caseids[run])]
        df = df.loc[df['case_id'].isin(runs_constrained_caseids[run])]
        df = df.sample(frac=1)
        train_size = int(0.75 * df.shape[0])
        test_size = df.shape[0] - train_size
        train = df.head(train_size)
        validate = df.tail(test_size)
        output_cols = ['days_to_death', 'days_to_followup']
        dtb_train = np.expand_dims(np.asarray(train['days_to_birth']), axis=1)
        mut_train = np.asarray([[float(y) if y != 'NA' else np.nan for y in x[1:-1].split(", ")] for x in list(train['mut_features'])])
        mut_train_load = np.sum(mut_train, axis=1).reshape(train.shape[0], 1)
        mut_train = mut_train[:, best_mut_feats]
        exp_train = np.asarray([[float(y) if y != 'NA' else np.nan for y in x[1:-1].split(", ")] for x in list(train['exp_features'])])
        x_train = np.asarray([[float(y) if y != 'NA' else np.nan for y in x[1:-1].split(", ")] for x in list(train['features'])])
        x_train = np.hstack((x_train, mut_train_load))
        x_train = np.hstack((x_train, dtb_train))
        x_train = np.hstack((x_train, exp_train))
        x_train = imp.transform(x_train)
        x_train = scaler.transform(x_train)
        x_train = np.hstack((x_train, mut_train))
        zero_mat = np.zeros((x_train.shape[0], len(disease_cols)))
        zero_mat[:, disease_idx] = np.ones(x_train.shape[0])
        x_train = np.hstack((x_train, zero_mat))
        days = np.asarray(train[output_cols])
        y_train = np.asarray([x[0] if np.isnan(x[0]) == False else x[1] for x in days])
        e_train = np.asarray([1 if np.isnan(x[0]) == False else 0 for x in days])
        y_train=nnet_survival.make_surv_array(y_train,[True if x == 1 else False for x in e_train],breaks)
        
        dtb_test = np.expand_dims(np.asarray(test['days_to_birth']), axis=1)
        mut_test = np.asarray([[float(y) if y != 'NA' else np.nan for y in x[1:-1].split(", ")] for x in list(test['mut_features'])])
        mut_test_load = np.sum(mut_test, axis=1).reshape(test.shape[0], 1)
        mut_test = mut_test[:, best_mut_feats]
        exp_test = np.asarray([[float(y) if y != 'NA' else np.nan for y in x[1:-1].split(", ")] for x in list(test['exp_features'])])
        x_test = np.asarray([[float(y) if y != 'NA' else np.nan for y in x[1:-1].split(", ")] for x in list(test['features'])])
        x_test = np.hstack((x_test, mut_test_load))
        x_test = np.hstack((x_test, dtb_test))
        x_test = np.hstack((x_test, exp_test))
        x_test = imp.transform(x_test)
        x_test = scaler.transform(x_test)
        x_test = np.hstack((x_test, mut_test))
        zero_mat = np.zeros((x_test.shape[0], len(disease_cols)))
        zero_mat[:, disease_idx] = np.ones(x_test.shape[0])
        x_test = np.hstack((x_test, zero_mat))
        days = np.asarray(test[output_cols])
        y_test = np.asarray([x[0] if np.isnan(x[0]) == False else x[1] for x in days])
        runs_disease_y_test_orig[(run, disease)] = y_test
        e_test = np.asarray([1 if np.isnan(x[0]) == False else 0 for x in days])
        y_test=nnet_survival.make_surv_array(y_test,[True if x == 1 else False for x in e_test],breaks)
        
        dtb_val = np.expand_dims(np.asarray(validate['days_to_birth']), axis=1)
        mut_val = np.asarray([[float(y) if y != 'NA' else np.nan for y in x[1:-1].split(", ")] for x in list(validate['mut_features'])])
        mut_val_load = np.sum(mut_val, axis=1).reshape(validate.shape[0], 1)
        mut_val = mut_val[:, best_mut_feats]
        exp_val = np.asarray([[float(y) if y != 'NA' else np.nan for y in x[1:-1].split(", ")] for x in list(validate['exp_features'])])
        x_val = np.asarray([[float(y) if y != 'NA' else np.nan for y in x[1:-1].split(", ")] for x in list(validate['features'])])
        x_val = np.hstack((x_val, mut_val_load))
        x_val = np.hstack((x_val, dtb_val))
        x_val = np.hstack((x_val, exp_val))
        x_val = imp.transform(x_val)
        x_val = scaler.transform(x_val)
        x_val = np.hstack((x_val, mut_val))
        zero_mat = np.zeros((x_val.shape[0], len(disease_cols)))
        zero_mat[:, disease_idx] = np.ones(x_val.shape[0])
        x_val = np.hstack((x_val, zero_mat))
        days = np.asarray(validate[output_cols])
        y_val = np.asarray([x[0] if np.isnan(x[0]) == False else x[1] for x in days])
        e_val = np.asarray([1 if np.isnan(x[0]) == False else 0 for x in days])
        y_val=nnet_survival.make_surv_array(y_val,[True if x == 1 else False for x in e_val],breaks)
        
        print(x_train.shape, x_val.shape, x_test.shape)
        
        if x_train.shape[1] != x_val.shape[1] or x_train.shape[1] != x_test.shape[1] or x_val.shape[1] != x_test.shape[1]:
            print("mismatched shape")
            continue

        if np.sum(e_train) < 0.3 * len(e_train) or np.sum(e_val) < 0.3 * len(e_val) or np.sum(e_test) < 0.3 * len(e_test) or len(e_train) < 150:
            print(np.sum(e_train), len(e_train), np.sum(e_val), len(e_val), np.sum(e_test), len(e_test))
            print("skipping ", disease)
            continue

        runs_disease_x_train[(run, disease)] = x_train
        runs_disease_x_test[(run, disease)] = x_test
        runs_disease_x_val[(run, disease)] = x_val
        runs_disease_y_train[(run, disease)] = y_train
        runs_disease_y_test[(run, disease)] = y_test
        runs_disease_y_val[(run, disease)] = y_val
        runs_disease_e_train[(run, disease)] = e_train
        runs_disease_e_test[(run, disease)] = e_test
        runs_disease_e_val[(run, disease)] = e_val
        runs_disease_n_intervals[(run, disease)] = n_intervals

In [ ]:
# Tuning for baseline models trained on global model weights

import keras
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Masking
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import optimizers
from keras import regularizers
import operator

runs_disease_trained_global_errors = {}
learning_rates = [0.1, 0.01, 0.001, 0.0001, 0.00001]
for run in range(10):
    for disease in [item[1] for item in runs_disease_x_train.keys() if item[0] == run]:
        for learning_rate in learning_rates:
            model = keras.models.clone_model(runs_global_models[run])
            model.set_weights(runs_global_models[run].get_weights())
            adam = optimizers.Adam(lr=learning_rate)
            model.compile(loss=nnet_survival.surv_likelihood(runs_disease_n_intervals[(run, disease)]), optimizer=adam)
            early_stopping = EarlyStopping(monitor='val_loss', patience=2)
            history = model.fit(runs_disease_x_train[(run, disease)], runs_disease_y_train[(run, disease)], batch_size=256, epochs=1000, shuffle=True,
                               callbacks=[early_stopping], validation_data=(runs_disease_x_val[(run, disease)], runs_disease_y_val[(run, disease)]))
            best_epochs = history.history['val_loss'].index(min(history.history['val_loss'])) + 1
            runs_disease_trained_global_errors[(run, (disease, learning_rate, best_epochs))] = min(history.history['val_loss'])

In [ ]:
# Train each transfer model using global model weights for initialization

runs_disease_trained_global_models = {}
for run in range(10):
    for disease in [item[1] for item in runs_disease_x_train.keys() if item[0] == run]:
        this_disease_runs = [x for x in runs_disease_trained_global_errors.keys() if disease == x[1][0] and run == x[0]]
        lowest_error = 100000
        best_run = None
        for disease_run in this_disease_runs:
            if runs_disease_trained_global_errors[disease_run] < lowest_error:
                lowest_error = runs_disease_trained_global_errors[disease_run]
                best_run = disease_run
        print(best_run, runs_disease_trained_global_errors[best_run])
        total_x = np.vstack((runs_disease_x_train[(run, disease)], runs_disease_x_val[(run, disease)]))
        total_y = np.vstack((runs_disease_y_train[(run, disease)], runs_disease_y_val[(run, disease)]))
        total_e = np.hstack((runs_disease_e_train[(run, disease)], runs_disease_e_val[(run, disease)]))
        model = keras.models.clone_model(runs_global_models[run])
        model.set_weights(runs_global_models[run].get_weights())
        adam = optimizers.Adam(lr=best_run[1][1])
        model.compile(loss=nnet_survival.surv_likelihood(runs_disease_n_intervals[(run, disease)]), optimizer=adam)
        model.fit(total_x, total_y, batch_size=256, epochs=best_run[1][2], shuffle=True)
        runs_disease_trained_global_models[(run, disease)] = model

In [ ]:
# Test each transfer model using global model weights for initialization

runs_disease_trained_global_loss = {}
for run in range(10):
    for disease in [item[1] for item in runs_disease_x_train.keys() if item[0] == run]:
        test_loss = runs_disease_trained_global_models[(run, disease)].evaluate(runs_disease_x_test[(run, disease)], runs_disease_y_test[(run, disease)])
        runs_disease_trained_global_loss[(run, disease)] = test_loss

In [ ]:
# Tuning for baseline models
# Find best hyperparameters for baseline model for each disease

from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Masking
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import optimizers
from keras import regularizers
import nnet_survival

patience = 2
runs_disease_baseline_errors = {}
units = [10, 50, 100, 200, 500, 1000]
learning_rates = [0.1, 0.01, 0.001, 0.0001, 0.00001]
for run in range(10):
    for disease in [item[1] for item in runs_disease_x_train.keys() if item[0] == run]:
        for unit in units:
            for learning_rate in learning_rates:
                print(run, disease, unit, learning_rate)
                num_epochs = 0
                min_loss = 1000000
                num_patience = 0
                model = Sequential()
                model.add(Dense(unit, activation='relu', input_shape=(runs_disease_x_train[(run, disease)].shape[1],)))
                model.add(Dense(runs_disease_n_intervals[(run, disease)], activation='sigmoid'))
                adam = optimizers.Adam(lr=learning_rate)
                model.compile(loss=nnet_survival.surv_likelihood(runs_disease_n_intervals[(run, disease)]), optimizer=adam)
                while True:
                    model.fit(runs_disease_x_train[(run, disease)], runs_disease_y_train[(run, disease)], batch_size=256, epochs=1, shuffle=True, verbose=0)
                    loss = model.evaluate(runs_disease_x_val[(run, disease)], runs_disease_y_val[(run, disease)], verbose=0)
                    if loss < min_loss:
                        num_epochs += 1
                        min_loss = loss
                        num_patience = 0
                    else:
                        if num_patience < patience:
                            num_epochs += 1
                            num_patience += 1
                        else:
                            break
                runs_disease_baseline_errors[(run, (disease, learning_rate, num_epochs - patience, unit))] = min_loss

In [ ]:
# Train each baseline model

runs_disease_baseline_models = {}
for run in range(10):
    for disease in [item[1] for item in runs_disease_x_train.keys() if item[0] == run]:
        this_disease_runs = [x for x in runs_disease_baseline_errors.keys() if disease == x[1][0] and run == x[0]]
        lowest_error = 100000
        best_run = None
        for disease_run in this_disease_runs:
            if runs_disease_baseline_errors[disease_run] < lowest_error:
                lowest_error = runs_disease_baseline_errors[disease_run]
                best_run = disease_run
        print(best_run, runs_disease_baseline_errors[best_run])
        total_x = np.vstack((runs_disease_x_train[(run, disease)], runs_disease_x_val[(run, disease)]))
        total_y = np.vstack((runs_disease_y_train[(run, disease)], runs_disease_y_val[(run, disease)]))
        total_e = np.hstack((runs_disease_e_train[(run, disease)], runs_disease_e_val[(run, disease)]))
        model = Sequential()
        model.add(Dense(best_run[1][3], activation='relu', input_shape=(total_x.shape[1],)))
        model.add(Dense(runs_disease_n_intervals[(run, disease)], activation='sigmoid'))
        adam = optimizers.Adam(lr=best_run[1][1])
        model.compile(loss=nnet_survival.surv_likelihood(runs_disease_n_intervals[(run, disease)]), optimizer=adam)
        model.fit(total_x, total_y, batch_size=256, epochs=best_run[1][2], shuffle=True)
        runs_disease_baseline_models[(run, disease)] = model

In [ ]:
# Test each baseline model

runs_disease_baseline_loss = {}
for run in range(10):
    for disease in [item[1] for item in runs_disease_x_train.keys() if item[0] == run]:
        test_loss = runs_disease_baseline_models[(run, disease)].evaluate(runs_disease_x_test[(run, disease)], runs_disease_y_test[(run, disease)])
        runs_disease_baseline_loss[(run, disease)] = test_loss

In [20]:
disease_diff = {}
for key in runs_disease_trained_global_loss.keys():
    diff = runs_disease_baseline_loss[key] - runs_disease_trained_global_loss[key]
    if key[1] not in disease_diff.keys():
        disease_diff[key[1]] = []
        disease_diff[key[1]].append(diff)
    else:
        disease_diff[key[1]].append(diff)
        
import plotly.plotly as py
import plotly.graph_objs as go

import numpy as np

data = [go.Box(y = disease_diff[x], name=x) for x in disease_diff.keys()]
py.iplot(data)